# **INTRODUCCIÓN**
Tomamos decisiones todos los días, grandes o pequeñas, correctas o incorrectas. Y cada decisión que tomamos, nos hace quienes somos.
En el año 1994, un joven de 30 años decidió dejar su lucrativo trabajo en Wall Street. Condujo por todo el país hasta Seattle y abrió una librería en línea. Lo llamó Amazonas. Y el joven es Jeff Bezos. Como reveló más tarde, tomó esta decisión debido a un informe que pronosticaba el rápido crecimiento de Internet, también por temor a arrepentirse más adelante en la vida.

Hoy, debido a la fiebre del oro de la IA, existe una necesidad urgente de talentos de TI calificados. Muchas mentes inteligentes, incluidos los trabajadores de Wall Street, por supuesto, pueden estar pensando en comenzar una nueva carrera en una de esas ciudades llenas de gigantes tecnológicos. Pero transferirse a una nueva carrera, mudarse a una nueva ciudad en sí ya es un gran desafío, por supuesto, esperamos que el nuevo vecindario al que nos mudamos sea similar al que hemos estado viviendo, para que no nos lleve demasiado tiempo y demasiado para adaptarse al nuevo entorno.

En este proyecto, estamos tratando de resolver este problema. Vamos a explorar la similitud entre los vecindarios de Manhattan y Seattle, tratando de encontrar algunas ideas como referencia sobre qué vecindario de Seattle debe elegir uno si se muda de un vecindario de Manhattan a Seattle.

Para completar esta tarea, vamos a utilizar los datos de ubicación de Foursquare para obtener las categorías de lugares más comunes en cada vecindario y luego, en base a esto, agruparemos los vecindarios en grupos con el algoritmo de aprendizaje automático de agrupación k-means, y presentaremos el resultado de la agrupación en un mapa generado con la biblioteca Folium.

Quizás se pregunte por qué Seattle, por qué no Silicon Valley. Bueno, no solo porque es la sede de Amazon (un miembro del club del billón de dólares), también es la sede de Microsoft (otro miembro del club del billón de dólares), Zillow, Tableau, Expedia, etc. Otros grandes empresas de tecnología como Google, Facebook, Twitter, todas tienen oficinas en Seattle. Además, el costo de vida, especialmente el precio de la vivienda, es muy amigable si se compara con el de Silicon Valley. Sin mencionar que el primer Starbucks en Pike Place podría ser una gran atracción para la diversión del café. En realidad, incluso Starbucks está utilizando activamente la IA, especialmente la ciencia de datos para fortalecer su poder. Entonces, si usted es uno de esos talentos, asegúrese de elegir a Seattle como una opción antes de tomar una decisión.

# **DATA**
Para este proyecto, necesitaremos recopilar el siguiente conjunto de datos:
- Datos del vecindario de Manhattan con coordenadas de latitud y longitud
- Datos del vecindario de Seattle con coordenadas de latitud y longitud
- Datos de ubicación de Foursquare: Categorías más comunes.

In [1]:
#importando librerías
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import requests
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import folium
%matplotlib inline

In [3]:
with open('ny_data.json') as json_data:
    ny_data = json.load(json_data)

In [7]:
neighborhood_data = ny_data['features']

In [9]:
neighborhood_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [10]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
ny_df = pd.DataFrame(columns = column_names)

In [11]:
for data in neighborhood_data:
    borough = data['properties']['borough']
    neighborhood = data['properties']['name'] 
    coordinates = data['geometry']['coordinates']
    latitude = coordinates[1]
    longitude = coordinates[0]
    ny_df = ny_df.append({'Borough': borough,
                        'Neighborhood': neighborhood,
                        'Latitude': latitude,
                        'Longitude': longitude}, ignore_index=True)
    

In [12]:
ny_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Recolectando data de Manhattan

In [13]:
mh_neigh = ny_df[ny_df.Borough == 'Manhattan'].reset_index(drop=True) 
mh_neigh.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
mh_neigh.rename(columns = {'Borough': 'Borough/City'}, inplace=True)
mh_neigh.head()

,Borough/City,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
mh_neigh['Borough/City'].value_counts()

Manhattan    40
Name: Borough/City, dtype: int64

In [17]:
size = mh_neigh.shape
print('El mh_neigh dataframe tiene {} filas y {} columnas.'.format(size[0], size[1]))

El mh_neigh dataframe tiene 40 filas y 4 columnas.


In [18]:
mh_neigh.isnull().sum()

Borough/City    0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [19]:
mh_neigh.duplicated().sum()

0

In [20]:
mh_neigh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Borough/City  40 non-null     object 
 1   Neighborhood  40 non-null     object 
 2   Latitude      40 non-null     float64
 3   Longitude     40 non-null     float64
dtypes: float64(2), object(2)
memory usage: 1.4+ KB


In [21]:
mh_neigh.to_csv('manhattan_neighborhood.csv')